In [ ]:
!git clone https://github.com/ozzux/DICA.git

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8m.yaml')

In [ ]:
results = model.train(data="/content/DICA/data.yaml", epochs=5)